##  감성체인 생성

In [17]:
import os
import time
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from concurrent.futures import ThreadPoolExecutor, as_completed
from langchain.chains import LLMChain  # LLMChain을 사용하여 Chain을 구성합니다
import logging

In [18]:
# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()

In [19]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# OpenAI LLM 모델 설정
llm = ChatOpenAI(openai_api_key=api_key, model_name="gpt-4-turbo-preview", temperature=0.3)

# 감성 분석 기본 프롬프트
prompt_template = PromptTemplate.from_template(
    """你是一名研究中国的社会科学家，你的任务是分析中国新闻对“美日韩”三边关系的报道，并从中国的视角分析这些报道的情感**。

    **评分标准**：
    - 强烈负面 (-5): 极端消极
    - 负面 (-3 ~ -4): 较为消极
    - 轻微负面 (-1 ~ -2): 略微消极
    - 中立 (0): 纯粹的事实陈述，不带有任何情感倾向。
    - 轻微正面 (+1 ~ +2): 略微积极
    - 正面 (+3 ~ +4): 较为积极
    - 强烈正面 (+5): 极端积极

    **请严格分析以下新闻文章的所有句子，并为每个句子提供一个情感分数。 句子应根据句号（。）进行分隔，不得遗漏任何句子**。
    **新闻文章内容:**
    {article_content}

    **请返回每个句子的情感分数，每行一个分数，不要包含任何额外的文本、注释或解释。**
    """
)

In [20]:
# 체인 구성
sentiment_chain = LLMChain(prompt=prompt_template, llm=llm)

## 데이터 로드

In [21]:
import json

# JSON 파일 로드
file_path = "../data/0310/10/split_by_year/2023.json"
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

print(f"총 {len(data)}개의 기사 로드 완료")


총 646개의 기사 로드 완료


## 감성분석 수행 함수

In [22]:
# 감성 분석 수행 함수
def analyze_sentiment(article):
    """기사 전체 본문을 입력하고 문장별 감성 점수를 분석하는 함수"""
    time.sleep(1.5)  # Rate Limit 방지를 위해 대기
    try:
        # 기사 본문만 전달
        input_data = {"article_content": article["cleaned_content"]}

        # `sentiment_chain.run()`을 사용하여 데이터 처리
        response = sentiment_chain.invoke(input_data)  # input_data 전달

        print(f"Response format: {response}")

        # 'text' 키에서 감성 분석 결과를 추출하여 'sentiment_responses'에 저장
        if isinstance(response, dict):
            if 'text' in response:
                article["sentiment_responses"] = response['text'].strip()  # 감성 분석 결과 저장
            else:
                article["sentiment_responses"] = "Error: 'text' key not found in response"
        else:
            article["sentiment_responses"] = "Error: Response is not a dictionary"

        return article
    except Exception as e:
        article["sentiment_responses"] = f"Error: {str(e)}"  # 오류 발생 시 오류 메시지 저장
        return article  # 오류 발생 시에도 응답 내용 저장

def process_article(article):
    """GPT 응답 후처리"""
    sentiment_response = article.get("sentiment_responses", "")

    # 문장이 없는 경우 바로 반환
    if not sentiment_response:
        return article

    sentiment_scores = []
    for line in sentiment_response.split("\n"):
        try:
            # 문장별 감성 점수를 처리
            sentiment_scores.append(float(line.strip()))  # 각 점수 추가
        except (ValueError, IndexError):
            continue
    article["sentiment_scores"] = sentiment_scores
    article["avg_sentiment_score"] = round(sum(sentiment_scores) / len(sentiment_scores), 2) if sentiment_scores else None
    return article

def run_sentiment_analysis(data, output_file):
    """JSON 파일을 로드하여 감성 분석을 병렬 실행하고 결과를 저장하는 함수"""
    results = []

    # 기사별 병렬 실행 (최대 3개 스레드 사용)
    with ThreadPoolExecutor(max_workers=3) as executor:
        future_to_article = {executor.submit(analyze_sentiment, article): article for article in data}

        for future in as_completed(future_to_article):
            article = future_to_article[future]  # 원본 기사 가져오기
            processed_article = process_article(article)  # 후처리
            if processed_article is not None:
                results.append(processed_article)  # 결과 저장

            # 중간 저장 (기사별로 즉시 JSON 파일 업데이트)
            with open(output_file, "w", encoding="utf-8") as f:
                json.dump(results, f, ensure_ascii=False, indent=4)
            print(f" '{article['title']}' 분석 완료.")
    print(f" 전체 기사 감성 분석 완료. {output_file}")

In [ ]:
output_path = "../data/0310/results/2023.json"
run_sentiment_analysis(data, output_path)

2025-03-10 11:58:42,735 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response format: {'article_content': '原标题郭晓兵亚太阵营对抗苗头值得警惕 有美国媒体日前刊文称亚洲已进入前所未见的大规模军备竞赛还说亚洲正因此分裂为美日韩和中俄朝两大阵营军备竞赛面临失控风险。在亚洲鼓噪阵营化和加剧对抗的问题上这篇文章说对一半美方确实试图下盘大棋。 一方面美国着力把亚太地区的轴辏式同盟体系改造为网络式同盟体系试图打造亚太小北约。美日韩是华盛顿着力经营的小多边体系之一。美去年出台的核态势评估报告提出要构建美日韩三边或美日韩澳四边延伸威慑磋商机制在亚太仿照北约编织多边核保护伞。日本则借机大幅调整防务政策准备大幅增加军费并发展导弹等非核战略进攻力量寻求实质性地突破专守防卫原则。但受制于日韩历史矛盾与利益分歧美日韩三边机制尚难成为美主导的亚太安全机制新支柱。 真正被美寄予厚望的恐怕是另外两个机制一是地理上环绕东亚大陆的美日印澳四边安全对话机制二是美英澳三边安全合作机制AUKUS同属盎格鲁撒克逊一系的三国在军事高科技分享与合作上更敢蹭红线美英计划帮助澳大利亚建造核潜艇并加强在高超音速武器攻防电子战网络空间人工智能量子技术等领域合作。另外美国2023国防授权法案授权向台提供100亿美元军援并在台建设区域应变军备库在亚太安全体系内继续以台制华意图明显。 与此同时美国还着力推进北约亚太化。这表现在两方面。一是把美国的亚太盟友拉进北约协调机制。去年北约邀请日韩澳新参加马德里峰会共商所谓欧亚协同。二是把英法德等北约欧洲成员国的军力派到亚太来。如果日后北约在乌克兰危机中承受的压力减小北约成员国在亚太地区可能会更活跃。 加速推动亚太阵营化说明美国更有信心还是更没有信心了呢？恐怕是后者。回顾历史美国充满自信的时候往往吆喝超越遏制和平演变接触扩展而在力有不逮并且内心焦虑的时候往往转而筹划拉拢盟国伙伴巩固基本盘之后寻觅时机徐图渐进。它在欧洲借乌克兰危机复兴北约就是出于这一目的在亚太借着炒作中国威胁再造联盟体系也是出于这一目的。 亚太阵营化将给地区带来什么？冲突和战乱。最新一期美国外交杂志有篇文章题目就是华盛顿对军事力量上瘾的根源指出冷战之后尤其是911事件以来美国的所谓对手并没频繁挑衅它但美国在使用武力方面却空前任性以至于全世界超过一半的人将美国视为威胁。美国如此黩武的原因之一就是与其他国家相比它拥有巨大军事优势。如果美国在亚太地区有了更加

2025-03-10 11:58:43,234 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response format: {'article_content': '原标题郭晓兵亚太阵营对抗苗头值得警惕 有美国媒体日前刊文称亚洲已进入前所未见的大规模军备竞赛还说亚洲正因此分裂为美日韩和中俄朝两大阵营军备竞赛面临失控风险。在亚洲鼓噪阵营化和加剧对抗的问题上这篇文章说对一半美方确实试图下盘大棋。 一方面美国着力把亚太地区的轴辏式同盟体系改造为网络式同盟体系试图打造亚太小北约。美日韩是华盛顿着力经营的小多边体系之一。美去年出台的核态势评估报告提出要构建美日韩三边或美日韩澳四边延伸威慑磋商机制在亚太仿照北约编织多边核保护伞。日本则借机大幅调整防务政策准备大幅增加军费并发展导弹等非核战略进攻力量寻求实质性地突破专守防卫原则。但受制于日韩历史矛盾与利益分歧美日韩三边机制尚难成为美主导的亚太安全机制新支柱。 真正被美寄予厚望的恐怕是另外两个机制一是地理上环绕东亚大陆的美日印澳四边安全对话机制二是美英澳三边安全合作机制AUKUS同属盎格鲁撒克逊一系的三国在军事高科技分享与合作上更敢蹭红线美英计划帮助澳大利亚建造核潜艇并加强在高超音速武器攻防电子战网络空间人工智能量子技术等领域合作。另外美国2023国防授权法案授权向台提供100亿美元军援并在台建设区域应变军备库在亚太安全体系内继续以台制华意图明显。 与此同时美国还着力推进北约亚太化。这表现在两方面。一是把美国的亚太盟友拉进北约协调机制。去年北约邀请日韩澳新参加马德里峰会共商所谓欧亚协同。二是把英法德等北约欧洲成员国的军力派到亚太来。如果日后北约在乌克兰危机中承受的压力减小北约成员国在亚太地区可能会更活跃。 加速推动亚太阵营化说明美国更有信心还是更没有信心了呢？恐怕是后者。回顾历史美国充满自信的时候往往吆喝超越遏制和平演变接触扩展而在力有不逮并且内心焦虑的时候往往转而筹划拉拢盟国伙伴巩固基本盘之后寻觅时机徐图渐进。它在欧洲借乌克兰危机复兴北约就是出于这一目的在亚太借着炒作中国威胁再造联盟体系也是出于这一目的。 亚太阵营化将给地区带来什么？冲突和战乱。最新一期美国外交杂志有篇文章题目就是华盛顿对军事力量上瘾的根源指出冷战之后尤其是911事件以来美国的所谓对手并没频繁挑衅它但美国在使用武力方面却空前任性以至于全世界超过一半的人将美国视为威胁。美国如此黩武的原因之一就是与其他国家相比它拥有巨大军事优势。如果美国在亚太地区有了更加

2025-03-10 11:58:46,898 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...

Response format: {'article_content': '环球时报综合报道当地时间8月18日美国总统拜登将在戴维营与日本首相岸田文雄和韩国总统尹锡悦举行美日韩三国领导人峰会。这是美日韩领导人在国际多边场合之外首次举行三方领导人峰会。对此美日韩政客和媒体一边宣称没有理由把这场峰会视为挑衅一边却欲盖弥彰地渲染三国共同研发高超音速导弹拦截系统商讨延伸核威慑强化所谓集体安全俨然要打造亚太小北约的架势。美国国家安全委员会印太事务协调员坎贝尔16日公开将矛头指向中国宣称这是华盛顿在回应亚太地区国家希望抵消中方压力的需求信号。这种拉小圈子搞阵营对抗的做法是企图在东北亚重新拉下冷战的铁幕。多名中国学者17日接受环球时报记者采访时表示美国紧拉日本和韩国为其充当马前卒找各种借口不断制造对抗和摩擦将使亚太地区相对稳定的局面被彻底破坏甚至重新蒙上战争的阴云。韩国最大在野党共同民主党国会代表朴洸瑥17日针对韩美日峰会表示如果站在遏制中国的最前线韩国就像站在刀刃上一样。 矛头指向中国 拜登政府高级官员说美日韩领导人计划在戴维营峰会上宣布加强三方安全关系的突破性举措包括分享有关中国和朝鲜弹道导弹威胁的情报以及定期举行联合军事演习等。美国全国广播公司NBC17日宣称这些前所未有的举动反映出对中国在该地区咄咄逼人的行动和朝鲜不断增长的导弹武器库的共同担忧为三国合作开启一个新的时代。 虽然美国国务院发言人帕特尔日前为美日韩峰会辩解称没有理由将这次峰会视为挑衅但美国政客和媒体近两天来处处将峰会的矛头直指中国。美国国家安全委员会印太事务协调员坎贝尔当地时间16日出席智库活动被问及此次峰会被称为亚太小北约时他并未否认这一说法还反咬一口称华盛顿是在回应亚太地区国家希望抵消中方压力的需求信号这些国家在经济外交和军事等许多方面都感受到了难以想象的巨大压力。当被问及峰会是否可能成为达成集体安全协议的第一步时坎贝尔称三国正在采取相当实质性的步骤来应对共同威胁。 路透社17日引述匿名美国官员的话透露称美日韩领导人将在峰会后发表联合声明启动一系列安全和军事方面的三方合作以强化集体安全。此外联合声明还将包括有关台海问题的内容。 辽宁大学美国与东亚研究院院长吕超17日接受环球时报记者采访时表示美国政客和媒体在美日韩峰会前释放出的信息基本透露了此次峰会的基调。美国要把日本和韩国更加紧密地绑在美国的战车上让日韩

2025-03-10 12:51:35,177 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response format: {'article_content': '美日韩领导人同框并不令人陌生但是本周五的戴维营之约与以往有很大不同。 这将是三国首脑第一次在多边场合之外单独举行会晤。 据悉三国首脑将讨论扩大三边合作朝鲜半岛局势经济安全等议题并将制定防御新步骤以增强集体安全。 外界评论称拜登政府力促美日韩首脑举行戴维营会谈意在利用日韩和解势头推动美日韩合作朝机制化方向发展以便三边关系能行稳致远从而应对来自中国朝鲜的所谓地区威胁。但是分析人士认为美日韩密切勾连的新动作可能才是地区和平稳定的威胁。 图片解放日报电子报8月16日第7版网页截屏部分 美国实现撮合夙愿 据日本媒体报道在美国举行三方对话是由美国总统拜登在今年的七国集团G7广岛峰会期间提出的。 美日韩领导人见个面并不难每年的亚太经合组织领导人会议东亚系列峰会甚至近年来的G7峰会北约峰会都为三国首脑会面提供良机。 美国为何非要跳出多边场合单独拉三国建个群开会呢？ 外界分析称一个很重要的考量是拜登政府希望趁热打铁利用日韩关系改善的外交红利进一步制度化深化和强化三边关系。 戴维营会晤将为过去一年一直在烘烤的三边关系蛋糕撒上新的糖霜。斯坦福大学国际政策和东亚研究讲师韩国经济研究院研究员丹尼尔斯奈德说。 中国现代国际关系研究院世界政治研究所所长研究员李岩认为美日韩首次单独举行首脑会晤而不是在多边场合这是前所未有的。这表明美国实现了夙愿把日韩撮合在一起使美日韩三边机制化变得现实可行。 在李岩看来以机制化推动三国关系发展是民主党政府一贯追求的做法其目的在于防范未来三国政府更迭导致前功尽弃。 上海社科院国际问题研究所研究员刘鸣认为将美日韩三边关系制度化的另一重考虑是与美英澳奥库斯美日印澳四方安全对话相配合在印太地区形成多层覆盖功能互补的复合型机制以达到推进印太战略的目的。 美日韩单独举行首脑会晤也与国际地区形势的变化密切相关。李岩说。首先美国在对华竞争的战略驱动下迫使中美关系发生重大变化。其次朝鲜半岛等地区安全问题已压过日韩矛盾。再次日韩政府目前对华态度与美国保持一致。这些因素都构成三国首脑首次单独会晤的重要背景。 至于会晤时机地点的选择也颇有讲究。 从时机来看美国认为现在三国单独举行一次首脑会谈已水到渠成。刘鸣说拜登上任以来美日韩领导人已在G7北约峰会等多边场合数次会晤美日美韩也各自举行了首脑会晤。更关键的因素是日韩

2025-03-10 12:51:37,670 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response format: {'article_content': '美日韩领导人同框并不令人陌生但是本周五的戴维营之约与以往有很大不同。 这将是三国首脑第一次在多边场合之外单独举行会晤。 据悉三国首脑将讨论扩大三边合作朝鲜半岛局势经济安全等议题并将制定防御新步骤以增强集体安全。 外界评论称拜登政府力促美日韩首脑举行戴维营会谈意在利用日韩和解势头推动美日韩合作朝机制化方向发展以便三边关系能行稳致远从而应对来自中国朝鲜的所谓地区威胁。但是分析人士认为美日韩密切勾连的新动作可能才是地区和平稳定的威胁。 图片解放日报电子报8月16日第7版网页截屏部分 美国实现撮合夙愿 据日本媒体报道在美国举行三方对话是由美国总统拜登在今年的七国集团G7广岛峰会期间提出的。 美日韩领导人见个面并不难每年的亚太经合组织领导人会议东亚系列峰会甚至近年来的G7峰会北约峰会都为三国首脑会面提供良机。 美国为何非要跳出多边场合单独拉三国建个群开会呢？ 外界分析称一个很重要的考量是拜登政府希望趁热打铁利用日韩关系改善的外交红利进一步制度化深化和强化三边关系。 戴维营会晤将为过去一年一直在烘烤的三边关系蛋糕撒上新的糖霜。斯坦福大学国际政策和东亚研究讲师韩国经济研究院研究员丹尼尔斯奈德说。 中国现代国际关系研究院世界政治研究所所长研究员李岩认为美日韩首次单独举行首脑会晤而不是在多边场合这是前所未有的。这表明美国实现了夙愿把日韩撮合在一起使美日韩三边机制化变得现实可行。 在李岩看来以机制化推动三国关系发展是民主党政府一贯追求的做法其目的在于防范未来三国政府更迭导致前功尽弃。 上海社科院国际问题研究所研究员刘鸣认为将美日韩三边关系制度化的另一重考虑是与美英澳奥库斯美日印澳四方安全对话相配合在印太地区形成多层覆盖功能互补的复合型机制以达到推进印太战略的目的。 美日韩单独举行首脑会晤也与国际地区形势的变化密切相关。李岩说。首先美国在对华竞争的战略驱动下迫使中美关系发生重大变化。其次朝鲜半岛等地区安全问题已压过日韩矛盾。再次日韩政府目前对华态度与美国保持一致。这些因素都构成三国首脑首次单独会晤的重要背景。 至于会晤时机地点的选择也颇有讲究。 从时机来看美国认为现在三国单独举行一次首脑会谈已水到渠成。刘鸣说拜登上任以来美日韩领导人已在G7北约峰会等多边场合数次会晤美日美韩也各自举行了首脑会晤。更关键的因素是日韩